In [1]:
import pandas as pd
import json
import numpy as np
from sklearn.model_selection import train_test_split
from simplet5 import *
import os
from utils import *

ddir = '../../data/'
dname = 'complete'
nbname = 't5-m_5\''
outputdir="/scratch/userid/" + nbname + '/'
if not os.path.exists(outputdir):
    os.makedirs(outputdir)
resultsfile = 'results/' + nbname + '.csv'
from sklearn.metrics import classification_report

Global seed set to 42


In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=2


env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=2


In [3]:
df_train = pd.read_csv(ddir + dname + '_train.csv')
df_test = pd.read_csv(ddir + dname + '_test.csv')
df_val = pd.read_csv(ddir + dname + '_val.csv')

In [4]:
def process_for_broad(df, mist, misl):
    if mist:
        df['mistype'] = df['mistype'].apply(lambda x: 'Simple' if (x == 'Converses' or x == 'Hypo-Hyper' or x == 'Mero-Holo') else x)
    if misl:
        df['mislocation'] = df['broadlocation']
    return df
df_train = process_for_broad(df_train, 1, 0)
df_test = process_for_broad(df_test, 1, 0)
df_val = process_for_broad(df_val, 1, 0)

In [5]:
def preprocess_df_m_1(df):
    X = []
    y = []
    ids = []
    for index, row in df.iterrows():
        if not isinstance(row['mislocation'], str) or not isinstance(row['mistype'], str) or row['mistype'] == 'Doubtful type' or row['mislocation'] == 'Doubtful location':
                continue
        try:
            tmp = ""
            tmp = "<claim> " + row['claim'] + "\n" if isinstance(row['claim'], str) else "<relation> "
            tmp += "<context> " + row['context'] + "\n" if isinstance(row['context'], str) else "<context> "
            tmp += "<source> " + row['Source'] + '\n' if isinstance(row['Source'], str) else "<source> "
            tmp += "<relation> " + row['Relation'] + '\n' if isinstance(row['Relation'], str) else "<relation> "
            tmp += "<target> " + row['Target'] + '\n' if isinstance(row['Target'], str) else "<target> " 
            tmp += "<mismatch> " + row['Mismatch'] + '\n' if isinstance(row['Mismatch'], str) else "<mismatch> " 
            tmp2 = "<mislocation> " + row['mislocation'] + "\n"
            tmp2 += "<mistype> " + row['mistype']
            X.append(tmp)
            y.append(tmp2)
            ids.append(row['id'])
        except:
            print(row)
            continue
    return X,y, ids

In [6]:
X_train, y_train, id_train = preprocess_df_m_1(df_train)
X_val, y_val, id_val = preprocess_df_m_1(df_val)
X_test, y_test, id_test = preprocess_df_m_1(df_test)

In [7]:
len(X_train), len(X_val), len(X_test)

(6443, 806, 806)

In [8]:
train_df = pd.DataFrame({'source_text': X_train, 'target_text': y_train, 'id': id_train})
val_df = pd.DataFrame({'source_text': X_val, 'target_text': y_val, 'id': id_val})
test_df = pd.DataFrame({'source_text': X_test, 'target_text': y_test, 'id': id_test})

In [9]:
train_df

source_text  \
0     <claim> Ederson Moraes plays only as a forward...   
1     <claim> A Song of Ice and Fire is by someone w...   
2     <claim> Beautiful couldn't chart on the Billbo...   
3     <claim> Jacqueline Kennedy Onassis is forgotte...   
4     <claim> Nikola Tesla didn't receive an educati...   
...                                                 ...   
6438  <claim> Inhumans is a movie.\n<context> Marvel...   
6439  <claim> Indian was solely produced by Mel Gibs...   
6440  <claim> Elle Fanning is not an actress.\n<cont...   
6441  <claim> The New Adventures of Old Christine is...   
6442  <claim> Chokher Bali was nominated for Best Ac...   

                                            target_text    id  
0        <mislocation> Target-Head\n<mistype> T Sisters  2145  
1        <mislocation> Target-Head\n<mistype> T Sisters  1719  
2        <mislocation> Target-Head\n<mistype> T Sisters  1079  
3         <mislocation> Relation-Head\n<mistype> Simple  5824  
4     <mislocation> Relation-Modifier\n<mistype> Neg...   712  
...                                                 ...   ...  
6438     <mislocation> Target-Head\n<mistype> T Sisters  4719  
6439     <mislocation> Target-Head\n<mistype> T Sisters  3022  
6440  <mislocation> Relation-Modifier\n<mistype> Neg...  5872  
6441     <mislocation> Target-Head\n<mistype> T Sisters  7052  
6442     <mislocation> Target-Head\n<mistype> T Sisters  5926  

[6443 rows x 3 columns]

In [10]:
# load model
model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-small")

/home/userid/anaconda/envs/p3/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [11]:
# train model
delete_whole_folder(outputdir)
model.train(train_df=train_df,
            eval_df=val_df, 
            source_max_token_len=256, 
            target_max_token_len=64, 
            batch_size=16, 
            max_epochs=4, 
            outputdir = outputdir,
            )
            


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [2]
Set SLURM handle signals.

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 60.5 M
-----------------------------------------------------
60.5 M    Trainable params
0         Non-trainable params
60.5 M    Total params
242.026   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/home/userid/anaconda/envs/p3/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 40 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
Global seed set to 42
/home/userid/anaconda/envs/p3/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 40 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [12]:
import os, fnmatch
def find(pattern, path):
    result = []
    for root, dirs, files in os.walk(path):
        for name in dirs:
            if fnmatch.fnmatch(name, pattern):
                result.append(os.path.join(root, name))
    return result
find('simplet5-epoch-3*', outputdir)[0]

"/scratch/userid/t5-m_5'/simplet5-epoch-3-train-loss-0.051-val-loss-0.0433"

In [13]:
from tqdm import tqdm
# load the best model

last_epoch_model = outputdir + find('simplet5-epoch-3*', outputdir)[0] # put the name here
model.load_model("t5-small", last_epoch_model, use_gpu=True)
# model.eval()
# for each test data perform prediction
predictions = []
for index, row in tqdm(test_df.iterrows()):
    prediction = model.predict(row['source_text'])[0]
    predictions.append(prediction)


# computer performance
test_df['predicted_label'] = predictions
test_df['original_label'] = test_df['target_text']

806it [03:00,  4.47it/s]


In [14]:
test_df

source_text  \
0    <claim> John Stewart is a comic book villain.\...   
1    <claim> Jimmy Carter served as President of Co...   
2    <claim> Christopher Lee portrayed Hillary Clin...   
3    <claim> Richard III of England died in the Bat...   
4    <claim> George H. W. Bush served as the 39th p...   
..                                                 ...   
801  <claim> Denial (2016 film) was released in the...   
802  <claim> Star Wars: The Force Awakens received ...   
803  <claim> Muhammad was raised under the care of ...   
804  <claim> Haim Saban is exclusively an Italian-A...   
805  <claim> Nikola Tesla refused to be a physicist...   

                                           target_text    id  \
0       <mislocation> Target-Head\n<mistype> T Sisters  4038   
1    <mislocation> Target-Modifier\n<mistype> T Sis...  5468   
2       <mislocation> Target-Head\n<mistype> T Sisters  4973   
3       <mislocation> Target-Head\n<mistype> T Sisters  6578   
4    <mislocation> Target-Modifier\n<mistype> Gradable   439   
..                                                 ...   ...   
801     <mislocation> Target-Head\n<mistype> T Sisters  3584   
802      <mislocation> Target-Head\n<mistype> Gradable   766   
803  <mislocation> Target-Modifier\n<mistype> T Sis...  2660   
804  <mislocation> Target-Modifier\n<mistype> T Sis...  1770   
805   <mislocation> Relation-Head\n<mistype> T Sisters  5413   

                                     predicted_label  \
0    mislocation> Target-Modifier mistype> T Sisters   
1        mislocation> Target-Head mistype> T Sisters   
2        mislocation> Target-Head mistype> T Sisters   
3        mislocation> Target-Head mistype> T Sisters   
4    mislocation> Target-Modifier mistype> T Sisters   
..                                               ...   
801      mislocation> Target-Head mistype> T Sisters   
802   mislocation> Target-Modifier mistype> Gradable   
803      mislocation> Target-Head mistype> T Sisters   
804  mislocation> Target-Modifier mistype> T Sisters   
805    mislocation> Relation-Head mistype> T Sisters   

                                        original_label  
0       <mislocation> Target-Head\n<mistype> T Sisters  
1    <mislocation> Target-Modifier\n<mistype> T Sis...  
2       <mislocation> Target-Head\n<mistype> T Sisters  
3       <mislocation> Target-Head\n<mistype> T Sisters  
4    <mislocation> Target-Modifier\n<mistype> Gradable  
..                                                 ...  
801     <mislocation> Target-Head\n<mistype> T Sisters  
802      <mislocation> Target-Head\n<mistype> Gradable  
803  <mislocation> Target-Modifier\n<mistype> T Sis...  
804  <mislocation> Target-Modifier\n<mistype> T Sis...  
805   <mislocation> Relation-Head\n<mistype> T Sisters  

[806 rows x 5 columns]

In [15]:
def process_output_r(x):
    x_split = x.split(' ')
    # ls = ['', '', '', '']
    ls = ['', '']
    itr = None
    for i in x_split:
        # if i == 'source>':
        #     itr = 0
        # elif i == 'relation>':
        #     itr = 1
        # elif i == 'target>':
        #     itr = 1
        if i == 'mislocation>':
            itr = 0
        elif i == 'mistype>':
            itr = 1
        elif itr is not None:
            ls[itr] += i + ' '
    return pd.Series([ls[0], ls[1]])


def process_input_r(x):
    x_split = x.split()
    # ls = ['', '', '', '']
    ls = ['', '']
    itr = None
    for i in x_split:
        # if i == 'source>':
        #     itr = 0
        # elif i == 'relation>':
        #     itr = 1
        # elif i == 'target>':
        #     itr = 1
        if i == '<mislocation>':
            itr = 0
        elif i == '<mistype>':
            itr = 1
        elif itr is not None:
            ls[itr] += i + ' '
    return pd.Series([ls[0], ls[1]])

In [16]:
test_df[['pMislocation', 'pMistype']] = test_df['predicted_label'].apply(process_output_r)
test_df[['Mislocation', 'Mistype']] = test_df['original_label'].apply(process_input_r)

In [17]:
# df_test['predicted_label'] = predictions
# df_test['original_label'] = test_df['target_text']

In [18]:
test_df

source_text  \
0    <claim> John Stewart is a comic book villain.\...   
1    <claim> Jimmy Carter served as President of Co...   
2    <claim> Christopher Lee portrayed Hillary Clin...   
3    <claim> Richard III of England died in the Bat...   
4    <claim> George H. W. Bush served as the 39th p...   
..                                                 ...   
801  <claim> Denial (2016 film) was released in the...   
802  <claim> Star Wars: The Force Awakens received ...   
803  <claim> Muhammad was raised under the care of ...   
804  <claim> Haim Saban is exclusively an Italian-A...   
805  <claim> Nikola Tesla refused to be a physicist...   

                                           target_text    id  \
0       <mislocation> Target-Head\n<mistype> T Sisters  4038   
1    <mislocation> Target-Modifier\n<mistype> T Sis...  5468   
2       <mislocation> Target-Head\n<mistype> T Sisters  4973   
3       <mislocation> Target-Head\n<mistype> T Sisters  6578   
4    <mislocation> Target-Modifier\n<mistype> Gradable   439   
..                                                 ...   ...   
801     <mislocation> Target-Head\n<mistype> T Sisters  3584   
802      <mislocation> Target-Head\n<mistype> Gradable   766   
803  <mislocation> Target-Modifier\n<mistype> T Sis...  2660   
804  <mislocation> Target-Modifier\n<mistype> T Sis...  1770   
805   <mislocation> Relation-Head\n<mistype> T Sisters  5413   

                                     predicted_label  \
0    mislocation> Target-Modifier mistype> T Sisters   
1        mislocation> Target-Head mistype> T Sisters   
2        mislocation> Target-Head mistype> T Sisters   
3        mislocation> Target-Head mistype> T Sisters   
4    mislocation> Target-Modifier mistype> T Sisters   
..                                               ...   
801      mislocation> Target-Head mistype> T Sisters   
802   mislocation> Target-Modifier mistype> Gradable   
803      mislocation> Target-Head mistype> T Sisters   
804  mislocation> Target-Modifier mistype> T Sisters   
805    mislocation> Relation-Head mistype> T Sisters   

                                        original_label      pMislocation  \
0       <mislocation> Target-Head\n<mistype> T Sisters  Target-Modifier    
1    <mislocation> Target-Modifier\n<mistype> T Sis...      Target-Head    
2       <mislocation> Target-Head\n<mistype> T Sisters      Target-Head    
3       <mislocation> Target-Head\n<mistype> T Sisters      Target-Head    
4    <mislocation> Target-Modifier\n<mistype> Gradable  Target-Modifier    
..                                                 ...               ...   
801     <mislocation> Target-Head\n<mistype> T Sisters      Target-Head    
802      <mislocation> Target-Head\n<mistype> Gradable  Target-Modifier    
803  <mislocation> Target-Modifier\n<mistype> T Sis...      Target-Head    
804  <mislocation> Target-Modifier\n<mistype> T Sis...  Target-Modifier    
805   <mislocation> Relation-Head\n<mistype> T Sisters    Relation-Head    

       pMistype       Mislocation     Mistype  
0    T Sisters       Target-Head   T Sisters   
1    T Sisters   Target-Modifier   T Sisters   
2    T Sisters       Target-Head   T Sisters   
3    T Sisters       Target-Head   T Sisters   
4    T Sisters   Target-Modifier    Gradable   
..          ...               ...         ...  
801  T Sisters       Target-Head   T Sisters   
802   Gradable       Target-Head    Gradable   
803  T Sisters   Target-Modifier   T Sisters   
804  T Sisters   Target-Modifier   T Sisters   
805  T Sisters     Relation-Head   T Sisters   

[806 rows x 9 columns]

In [19]:
def exact_match(orig, predicted):
    match = 0
    for i in range(len(orig)):
        if orig[i].strip() == predicted[i].strip():
            # print(orig[i], "&", predicted[i])
            match += 1
    return match/len(orig)

def partial_match(orig, predicted):
    match = 0
    for i in range(len(orig)):
        orig_split = orig[i].split()
        predicted_split = predicted[i].split()
        if len(orig_split) == 0:
            if len(predicted_split) == 0:
                match += 1
            else:
                match += 0
            continue
        ints = list(set(orig_split) & set(predicted_split))
        match += len(ints)/len(orig_split)
    return match/len(orig)

In [20]:
# exact_match(list(test_df.pSource), list(test_df.Source)), partial_match(list(test_df.pSource), list(test_df.Source))

In [21]:
# exact_match(list(test_df.pRelation), list(test_df.Relation)), partial_match(list(test_df.pRelation), list(test_df.Relation))

In [22]:
# exact_match(list(test_df.pTarget), list(test_df.Target)), partial_match(list(test_df.pTarget), list(test_df.Target))

In [23]:
exact_match(list(test_df.Mislocation), list(test_df.pMislocation)), partial_match(list(test_df.Mislocation), list(test_df.pMislocation))

(0.7841191066997518, 0.7841191066997518)

In [24]:
exact_match(list(test_df.Mistype), list(test_df.pMistype)), partial_match(list(test_df.Mistype), list(test_df.pMistype))

(0.7518610421836228, 0.7518610421836228)

In [25]:
test_df.to_csv(resultsfile)

In [26]:
print(classification_report(test_df.Mislocation, test_df.pMislocation))

                    precision    recall  f1-score   support

    Relation-Head        0.80      0.62      0.70        91
Relation-Modifier        0.83      0.92      0.87       154
      Source-Head        0.00      0.00      0.00         7
  Source-Modifier        0.50      0.67      0.57         3
      Target-Head        0.81      0.88      0.84       398
  Target-Modifier        0.66      0.54      0.59       153

          accuracy                           0.78       806
         macro avg       0.60      0.60      0.60       806
      weighted avg       0.77      0.78      0.78       806



In [27]:
print(classification_report(test_df.Mistype, test_df.pMistype))

              precision    recall  f1-score   support

   Gradable        0.55      0.45      0.49        53
   Negation        0.82      0.93      0.87       163
  Set Based        0.57      0.70      0.63        64
     Simple        0.18      0.24      0.20        42
  T Sisters        0.85      0.78      0.81       484

    accuracy                           0.75       806
   macro avg       0.59      0.62      0.60       806
weighted avg       0.77      0.75      0.76       806



In [28]:
join_df = test_df[["id","pMistype","pMislocation"]]
res_df = df_test.join(join_df.set_index('id'), on='id')
res_df.to_csv(resultsfile)

In [29]:
res_df.columns

Index(['Unnamed: 0', 'id', 'claim', 'context', 'Source', 'SSpan', 'Relation',
       'RSpan', 'Target', 'TSpan', 'mislocation', 'Mismatch', 'MSpan',
       'mistype', 'half_clustered_entity', 'clustered_entity_filtered',
       'split_column', 'pMistype', 'pMislocation'],
      dtype='object')

In [30]:
test_df = pd.read_csv("results/t5-m3.csv")

In [31]:
test_df = test_df.drop(['Source', 'Relation', 'Target', 'Mismatch'], axis = 1)
test_df.rename(columns = {'pSource': 'Source', 'pRelation': 'Relation', 'pTarget': 'Target', 'pMismatch': 'Mismatch'}, inplace=True)
X_test, y_test, id_test = preprocess_df_m_1(test_df)
test_df = pd.DataFrame({'source_text': X_test, 'target_text': y_test, 'id': id_test})

In [32]:
test_df

source_text  \
0    <claim> John Stewart is a comic book villain.\...   
1    <claim> Jimmy Carter served as President of Co...   
2    <claim> Christopher Lee portrayed Hillary Clin...   
3    <claim> Richard III of England died in the Bat...   
4    <claim> George H. W. Bush served as the 39th p...   
..                                                 ...   
801  <claim> Denial (2016 film) was released in the...   
802  <claim> Star Wars: The Force Awakens received ...   
803  <claim> Muhammad was raised under the care of ...   
804  <claim> Haim Saban is exclusively an Italian-A...   
805  <claim> Nikola Tesla refused to be a physicist...   

                                           target_text    id  
0       <mislocation> Target-Head\n<mistype> T Sisters  4038  
1    <mislocation> Target-Modifier\n<mistype> T Sis...  5468  
2       <mislocation> Target-Head\n<mistype> T Sisters  4973  
3       <mislocation> Target-Head\n<mistype> T Sisters  6578  
4    <mislocation> Target-Modifier\n<mistype> Gradable   439  
..                                                 ...   ...  
801     <mislocation> Target-Head\n<mistype> T Sisters  3584  
802      <mislocation> Target-Head\n<mistype> Gradable   766  
803  <mislocation> Target-Modifier\n<mistype> T Sis...  2660  
804  <mislocation> Target-Modifier\n<mistype> T Sis...  1770  
805   <mislocation> Relation-Head\n<mistype> T Sisters  5413  

[806 rows x 3 columns]

In [33]:
from tqdm import tqdm
# load the best model

last_epoch_model = outputdir + find('simplet5-epoch-3*', outputdir)[0] # put the name here
model.load_model("t5-small", last_epoch_model, use_gpu=True)
# model.eval()
# for each test data perform prediction
predictions = []
for index, row in tqdm(test_df.iterrows()):
    prediction = model.predict(row['source_text'])[0]
    predictions.append(prediction)


# computer performance
test_df['predicted_label'] = predictions
test_df['original_label'] = test_df['target_text']

806it [03:06,  4.33it/s]


In [34]:
test_df[['pMislocation', 'pMistype']] = test_df['predicted_label'].apply(process_output_r)
test_df[['Mislocation', 'Mistype']] = test_df['original_label'].apply(process_input_r)

In [35]:
print(classification_report(test_df.Mislocation, test_df.pMislocation))

                    precision    recall  f1-score   support

    Relation-Head        0.62      0.58      0.60        91
Relation-Modifier        0.82      0.92      0.87       154
      Source-Head        0.00      0.00      0.00         7
  Source-Modifier        0.67      0.67      0.67         3
      Target-Head        0.79      0.87      0.83       398
  Target-Modifier        0.60      0.41      0.49       153

          accuracy                           0.75       806
         macro avg       0.58      0.57      0.57       806
      weighted avg       0.73      0.75      0.74       806



/home/userid/anaconda/envs/p3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/userid/anaconda/envs/p3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/userid/anaconda/envs/p3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [36]:
print(classification_report(test_df.Mistype, test_df.pMistype))

              precision    recall  f1-score   support

   Gradable        0.49      0.45      0.47        53
   Negation        0.82      0.93      0.87       163
  Set Based        0.51      0.41      0.45        64
     Simple        0.17      0.24      0.20        42
  T Sisters        0.83      0.80      0.82       484

    accuracy                           0.74       806
   macro avg       0.56      0.56      0.56       806
weighted avg       0.75      0.74      0.74       806

